# Canal plan 2D : comparaison des méthodes numériques


In [ ]:
from trustutils import run 
from trustutils.jupyter import plot
import numpy as np
from functions import *
from math import *
import matplotlib.pyplot as plt

run.introduction("M. El Moatamid","22/05/2023")
run.description(description_fiche)

run.TRUST_parameters()

# A remplir par l'utilisateur 

In [ ]:
# # Configuration

# Liste des configuration disponible : ["VEF_k-epsilon", "VDF_k-epsilon", "VDF_k-tau", "VDF_k-omega", "PolyMAC_k-tau", "PolyMAC_k-omega", "PolyMAC_k-tau_triangle"]
config = ["VEF_k-epsilon", "VDF_k-epsilon", "VDF_k-tau", "VDF_k-omega", "PolyMAC_k-tau", "PolyMAC_k-omega"]
# Il est malheureusement important de garder la liste config dans cet ordre

# # Parallelisation du calcul

number_of_partitions = 8

# # Fluide

mu = 0.02
rho = 1000

# # Conditions limites

inlet_velocity = 1
outlet_pressure = 1e5
inlet_k = 0.01
inlet_epsilon = 0.005

# # Maillage

# Attention il y aura 2 fois plus de cellules selon y que la valeur définie ici (trianguler_h avec vef coupe les maille en 4)
Ny = [2,3,4] # nombre de noeuds dans le 1er bloc de maillage VEF
taux1 = [1,1,1] # facteur multiplicatif : taille des cellules dans le bloc n°2
taux2 = [1,1,1] # facteur multiplicatif : taille des cellules dans le bloc n°3

# # Boucle temporelle

tmax = 300 # [s]

# # Post traitement

# plot profiles
x_prof = 95 # position selon x pour extraire différents profils (vitesse, k, ...)
y_min_prof = sonde_firstpoint(0.2,Ny)
y_max_prof = 1-sonde_firstpoint(0.2,Ny)
nb_points_prof = sonde_nbpoints(0.2,Ny)

### Premier calculs sur les paramètres entrés
On calcule sur la base des paramètres de simulation choisis par l'utilisateur :
- Le nombre de Reynolds
- Une estimation du coef de frottement qui permet de prédire (bonne approximation) tau_w, utau et y⁺

L'utilisateur peut donc savoir à quels résultats s'attendre et vérifier que les valeurs de y+ permettent d'utiliser une loi de paroi log (loi standard utilisée avec VEF)
    
On vérifie également que le maillage peut être construit

In [ ]:
H = 1 # hauteur canal [m]
Re_H = Reynolds(inlet_velocity, rho, H, mu) # calcul du reynolds
cf_pred = cf_prediction(kappa, B, Re_H, cf_guess) # estimation du coefficient de frottement
tau_pred, u_tau_pred = tau_and_u_tau(cf_pred,1,1000) # tau et u_tau estimés
print(f"Reynolds : Re_H = {Re_H:.1e}\n\nEstimations :\n\tcf = {cf_pred:.4f} \ttau_w = {tau_pred:.2f} Pa/s\tu_tau = {u_tau_pred:.3f} m/s")
print("\nMaillages :\n Valeurs estimées de y+ à la paroi :\n")
y_wall_list = [[0 for x in range(len(Ny))] for y in range(3)] # hauteur maille à la paroi
y_plus_list = [[0 for x in range(len(Ny))] for y in range(3)] # y+ paroi
for i in range(len(Ny)):
    y_VDF, y_VEF, y_PolyMAC = y_wall(y1,Ny[i]) # valeurs de y à l'endroit où la vitesse à la paroi est calculée
    y_wall_list[0][i], y_wall_list[1][i], y_wall_list[2][i] = y_VEF, y_VDF, y_PolyMAC
    y_VDF_plus, y_VEF_plus, y_PolyMAC_plus = y_plus_prediction(y1,Ny[i],u_tau_pred,mu,rho) # y+ correspondant à y_wall calculé
    y_plus_list[0][i], y_plus_list[1][i], y_plus_list[2][i] = y_VEF_plus, y_VDF_plus, y_PolyMAC_plus
    print(f" - mesh_{i} : y+_VDF = {y_VDF_plus:.1f}\t y+_VEF = {y_VEF_plus:.1f}\t y+_PolyMAC = {y_PolyMAC_plus:.1f}")
    check_mesh_param(Ny[i],taux1[i],taux2[i])

## 1- Initialisation et Lancement du calcul
### Initialisation

In [ ]:
force_recalculation = False # put True to force recalculation
build = run.BUILD_DIRECTORY # build directory (absolute path !)
method_Pb_hydr, method_Pb_multi = method_name(config) # liste des noms de discretisations

In [ ]:
nb_mesh = len(Ny)
nb_method_Pb_hydr = len(method_Pb_hydr)
nb_method_Pb_multi = len(method_Pb_multi)
nb_method = nb_method_Pb_hydr + nb_method_Pb_multi
# paramètres utilisés pour la substitution : voir function.py
params = [config, Ny, taux1, taux2, method_Pb_hydr, method_Pb_multi, inlet_velocity,outlet_pressure,inlet_k,inlet_epsilon,x_prof,mu,rho,y_min_prof,y_max_prof,nb_points_prof,tmax]
# # Substitution dictionnary : nous avons 2 jdd de base qui seront modifiés pour lancer les différentes config
dict_list_hydr = substitution_pb_hydr(params)
dict_list_multi = substitution_pb_multi(params)
dict_list = [[0 for x in range(nb_mesh)] for y in range(nb_method)]
for i in range(nb_method_Pb_hydr):
    for j in range(nb_mesh):
        dict_list[i][j] = dict_list_hydr[i][j]
for i in range(nb_method_Pb_multi):
    for j in range(nb_mesh):
        dict_list[i+2][j] = dict_list_multi[i][j] # ATTENTION : indice i+2 car les 2 premières config sont Pb_hydraulique

# data file names 
file_pb_hydr = "canal_plan_pb_hydr"
file_pb_multi = "canal_plan_pb_multi"

import os  
if force_recalculation or not os.path.exists(build):
    run.reset() # Delete the build directory and empty the list of cases to be executed
    run.defaultSuite_ = run.TRUSTSuite(runPrepare=False)
    for i in range(nb_method):
        file = file_pb_hydr
        if i>1: # car les 2 premières config sont Pb_hydraulique
            file = file_pb_multi
        for j in range(nb_mesh):
            if number_of_partitions == 1:
                run.addCaseFromTemplate(f"{file}.data",directory=build+f"/{config[i]}/mesh_{j}",d=dict_list[i][j])
            else:
                os.system(f"mkdir {build}/{config[i]}")
                os.system(f"mkdir {build}/{config[i]}/mesh_{j}")
                GenerateInputFile(f"{build}/{config[i]}/mesh_{j}",build,dict_list[i][j], file)
                os.system(f'cd {build}/{config[i]}/mesh_{j};  make_PAR.data {file} {number_of_partitions}; cd ../../..')
                run.addCase(f"{config[i]}/mesh_{j}", f"PAR_{file}.data",nbProcs=number_of_partitions)
    run.printCases()
    run.runCases() # lance les différents cas test écrits dans build/
    perf = run.tablePerf() # tableau des performances de calcul
else:
    print("Using previous calculation")
    
if number_of_partitions!=1: # nom des jdd change pour les calculs parallèles
    file_pb_multi = "PAR_canal_plan_pb_multi"
    file_pb_hydr = "PAR_canal_plan_pb_hydr"

In [ ]:
# Tableau des performances de calcul
import pandas as pd

performance = open(build+"/perf.csv", "w")
for i in range(nb_method):
    file = file_pb_multi
    if "k-epsilon" in config[i]: # car les config avec "k-epsilon" sont les seuls config avec pb_hydraulique
        file = file_pb_hydr
    for j in range(nb_mesh):
        performance.write(read_perf(build+f"/{config[i]}/mesh_{j}/{file}.perf", file, config[i]+f" mesh_{j}"))
performance.close() 
performance = open(build+"/perf.csv", "r")
data = pd.read_csv(performance, sep=" ", header=None)
df = data[data.columns[:-1]]
df.columns = ["Configuration", "Mesh", "Host", "System", "total_CPU_time", "CPU time/step", "nb_cells"]
blankIndex=[''] * len(df)
df.index=blankIndex
display(df)

In [ ]:
# Graph temps de calcul
grpd = df.groupby('Configuration')
for name, data in grpd:
    plt.plot(data.nb_cells.values, data.total_CPU_time.values, 'o-', label = name)
plt.legend()
plt.xlabel("number of cells")
plt.ylabel("Total CPU time [s]")
plt.title("Calculation time")

### Résidus

In [ ]:
# Residuals Plot
fig = plt.figure(figsize=(15,ceil(nb_method/3)*4))
axs = fig.subplots(ceil(nb_method/3),3)


for i in range(nb_method):
    file = file_pb_hydr
    if i>1: # car les 2 premières config sont Pb_hydraulique
        file = file_pb_multi
    for j in range(nb_mesh):
        res_file = build+f"/{config[i]}/mesh_{j}/{file}.dt_ev"
        max_residu = plot.loadText(res_file)[3,:]
        # find what is the max residual (called res_name)
        res_name = ""
        for k, res in enumerate(plot.loadText(res_file)[4:,-1]):
            if max_residu[-1] == res:
                res_name = get_residu_name(res_file,k)
        temps = plot.loadText(res_file)[0,:]
        axs[int(i/3),i%3].plot(temps, max_residu,label=f"mesh_{j}")
        axs[int(i/3),i%3].text(0.3, 0.3-j*0.1, f'mesh_{j} : {res_name}', horizontalalignment='center', verticalalignment='center', transform=axs[int(i/3),i%3].transAxes)
    axs[int(i/3),i%3].legend(fontsize=10)
    axs[int(i/3),i%3].set_xlabel("time", fontsize=10)
    axs[int(i/3),i%3].set_ylabel("maxResidu", fontsize=10)
    axs[int(i/3),i%3].set_yscale('log')
    axs[int(i/3),i%3].set_title(f'{config[i]}', fontsize=12)
fig.suptitle(f"Residuals")
fig.tight_layout()

### Maillage
décommenter pour voir le maillage

In [ ]:
#from trustutils import visit
#for i in range(nb_method_Pb_hydr):
#    visit.showMesh(build+f"/{config[i]}/mesh_0/{file_pb_hydr}.lata","dom")
#for i in range(nb_method_Pb_multi):
#    visit.showMesh(build+f"/{config[i+2]}/mesh_0/{file_pb_multi}.lata","dom")

### Lire quelques résultats
Vérifier que la couche limite est bien développée : tracé de tau_wall(x)

In [ ]:
# # Calcul de u_tau (noté u*) pour les différents maillages
# # tracé de tau_w le long de la paroi : vérifier que la couche limite est bien developpée

print(f"Calcul des valeurs de u_tau à x={x_prof}m ainsi que y+ à la fontière sup de la 1ère maille pour VDF et PolyMAC et à la hauteur des centres des faces pour VEF :\n\n")
u_star_list = [[0 for x in range(nb_mesh)] for y in range(nb_method)] # liste des u_tau calculés à x_prof
y_plus_cal_list = [[0 for x in range(nb_mesh)] for y in range(nb_method)] # liste des y+ calculés à x_prof

# valeur min et max de tau (sert à mettre toutes les courbes à la meme échelle)
min_tau = 50 
max_tau = 0 

fig = plt.figure(figsize = (15,4*ceil(nb_method/2)))
axs = fig.subplots(ceil(nb_method/2),2)
for i in range(nb_method):
    for j in range(nb_mesh):
        if i<2: # avec pb_hydraulique : tau_w déduit des données dans le fichier Ustar.face
            # plot tau_x=f(x)
            wall_file = build+f'/{config[i]}/mesh_{j}/{file_pb_hydr}_pb_Ustar.face'
            data = read_facefile(wall_file)
            keys = list(data.keys())
            x, y, u_plus, d_plus, u_star, tau, tau_x, tau_y = get_wall_param(data[keys[-1]])
            tau_x = rho*tau_x
            # save value of u_tau
            u_plus, d_plus, u_star, tau, tau_xprof, tau_yprof = get_values_at_x(data[keys[-1]], x_prof)
            u_star_list[i][j] = u_star
            y_plus = y_wall_list[i][j]*u_star*rho/mu
            y_plus_cal_list[i][j] = y_plus
            print(f"{config[i]}\t, mesh_{j} :\t u* = {u_star:.4f} m/s,\t\t y+={y_plus:.1f}")
        else: # avec pb_hmultiphase : tau_w déduit de la sonde de y+
            # plot tau_x=f(x)
            x, y_demi = abscisses_sonde_yplus(config[i],Ny[j])
            y_plus = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file_pb_multi}_Y_PLUS.son")[1::,1])
            u_tau, tau_x = get_tau_from_yplus(y_plus, y_demi, rho, mu)
            # save value of u_tau
            u_star = get_value_at_x(u_tau,x,x_prof)
            u_star_list[i][j] = u_star
            y_plus = 2*y_demi*u_star*rho/mu
            y_plus_cal_list[i][j] = y_plus
            print(f"{config[i]}\t, mesh_{j} :\t u* = {u_star:.4f} m/s,\t\t y+={y_plus:.1f}") # 2*y_demi pour etre à la frontière sup de la 1ere maille à la paroi
        if min(tau_x)<min_tau:
            min_tau = min(tau_x)
        if max(tau_x)>max_tau:
            max_tau = max(tau_x)    
        axs[int(i/2),i%2].plot(x, tau_x, label=f"mesh_{j}") # Change plotted variable here (choose u_plus, d_plus, u_star, tau, tau_x or tau_y)
    axs[int(i/2),i%2].plot(x,tau_pred*np.ones(len(x)), "--",label='estimation')
    axs[int(i/2),i%2].legend(fontsize=12)
    axs[int(i/2),i%2].set_xlabel("x [m]", fontsize=10)
    axs[int(i/2),i%2].set_ylabel("tau_x [Pa/s]", fontsize=10)
    axs[int(i/2),i%2].set_title(f'{config[i]}', fontsize=12)

# Defining custom 'xlim' and 'ylim' values.
custom_xlim = (0, 100)
custom_ylim = (min_tau-0.1, max_tau+0.1)

# Setting the values for all axes.
plt.setp(axs, xlim=custom_xlim, ylim=custom_ylim)

fig.suptitle(f"Contraintes pariétales à t={round(keys[-1],1)}s")
fig.tight_layout()
print(f"\nComparaison avec la estimation : u* = {u_tau_pred:.4f} m/s")
for i in range(nb_mesh):
    print(f" - mesh_{i} y+ à la paroi : y+_VEF = {y_plus_list[0][i]:.1f}\t y+_VDF = {y_plus_list[1][i]:.1f}\t y+_PolyMAC = {y_plus_list[2][i]:.1f}")

## 2 - Afficher les resultats
### 2.1 - Profils de vitesse

#### Etude de la convergence en maillage

In [ ]:
# # Plot u⁺=f(y⁺)
fig = plt.figure(figsize = (15,4*ceil(nb_method/2)))

axs = fig.subplots(ceil(nb_method/2),2)

for i in range(nb_method):
    file = file_pb_hydr
    if i>1:
        file = file_pb_multi
    for j in range(nb_mesh):
        utau = u_star_list[i][j] 
        uplus = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_VITESSE.son")[1::2,1])/utau
        yplus = np.linspace(y_min_prof[j], y_max_prof[j], nb_points_prof[j])*utau/(mu/rho)
        axs[int(i/2),i%2].plot(yplus, uplus, "-o",markersize=3,label=f"mesh_{j}: u⁺=f(y⁺)")
        axs[int(i/2),i%2].plot(y_plus_cal_list[i][j]*np.ones(2),np.linspace(4,20,2), linestyle ='dotted', label=f"mesh_{j}: y⁺wall={y_plus_cal_list[i][j]:.0f}")
    axs[int(i/2),i%2].plot(yplus[yplus>10], (1/kappa)*np.log(yplus[yplus>10])+B, "--", label="log law", color="black")
    #axs[int(i/2),i%2].plot(yplus[yplus<15], yplus[yplus<15], "--", label="u⁺ = y⁺", color="grey")
    axs[int(i/2),i%2].legend(fontsize=10)
    axs[int(i/2),i%2].set_xlabel("y+", fontsize=12)
    axs[int(i/2),i%2].set_ylabel("u+", fontsize=12)
    axs[int(i/2),i%2].set_xscale('log')
    axs[int(i/2),i%2].set_title(f'{config[i]}', fontsize=12)
fig.suptitle(f"Profils de vitesse à x={x_prof}m : convergence en maillage")
fig.tight_layout()

In [ ]:
# # Plot u=f(y)

Graph=plot.Graph("Velocity profiles : convergence en maillage", nX=ceil(nb_method/2),nY=2,size=[8,6])
for i in range(nb_method):
    file = file_pb_hydr
    if i>1:
        file = file_pb_multi
    Graph.addPlot((int(i/2),i%2),f"{config[i]}")
    for j in range(nb_mesh):
        Graph.addSegment(build+f"/{config[i]}/mesh_{j}/{file}_VITESSE.son", linestyle="solid", marker = 'o', markersize=3,compo=0,label=f"mesh_{j}")
    Graph.legend()
    Graph.label("y [m]", "u [m/s]")

#### Comparaison des différentes méthodes de calcul

In [ ]:
# # Plot u+=f(y+)

Graph=plot.Graph("Velocity profiles : comparaison des méthodes", nY=nb_mesh,size=[8,5])
for j in range(nb_mesh):
    Graph.addPlot(j,f"mesh_{j}")
    for i in range(nb_method):
        file = file_pb_multi
        utau = u_tau_pred
        if i<2:
            utau = u_star_list[i][j] 
            file = file_pb_hydr
        uplus = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_VITESSE.son")[1::2,1])/utau
        yplus = np.linspace(y_min_prof[j], y_max_prof[j], nb_points_prof[j])*utau/(mu/rho)
        Graph.add(yplus, uplus, linestyle="solid", marker = 'o', markersize=3,label=f"{config[i]}")
    Graph.add(yplus[yplus>10], (1/kappa)*np.log(yplus[yplus>10])+B, "--", linewidth=2, label="log law", color="black")
    Graph.scale(xscale='log')
    Graph.legend()
    Graph.label("y⁺", "u⁺")

In [ ]:
# # Plot u=f(y)

Graph=plot.Graph("Velocity profiles : comparaison des méthodes", nY=nb_mesh,size=[8,6])
for j in range(nb_mesh):
    Graph.addPlot(j,f"mesh_{j}")
    for i in range(nb_method):
        file = file_pb_hydr
        if i>1:
            file = file_pb_multi
        Graph.addSegment(build+f"/{config[i]}/mesh_{j}/{file}_VITESSE.son",compo=0, linestyle="solid", marker = 'o', markersize=3,label=f"{config[i]}")
    Graph.legend()
    Graph.label("y [m]", "u [m/s]")

### 2.2 - Traceurs de la turbulence
#### Etude de la convergence en maillage

In [ ]:

Graph=plot.Graph(title="Traceurs de turbulence : convergence en maillage",nY=2,nX=nb_method,size=[8,6])

for i in range(nb_method):
    file = file_pb_hydr
    if i>1:
        file = file_pb_multi
    Graph.addPlot((i,0),f"{config[i]}")
    for j in range(nb_mesh):
        Graph.addSegment(build+f"/{config[i]}/mesh_{j}/{file}_K.son", linestyle="solid", marker = 'o', markersize=3,label=f"mesh_{j}")
    Graph.legend()
    Graph.label("y [m]", "k [m²/s²]")


for i in range(nb_method):
    file = file_pb_hydr
    if i>1:
        file = file_pb_multi
    Graph.addPlot((i,1),f"{config[i]}")
    for j in range(nb_mesh):
        diss = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_DISS.son")[1::,1])
        k = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_K.son")[1::,1])
        if "tau" in config[i]:
            epsilon = 0.09*k/diss
        elif "omega" in config[i]:
            epsilon = 0.09*k*diss
        else:
            epsilon = diss
        y = np.linspace(y_min_prof[j], y_max_prof[j], nb_points_prof[j])
        Graph.add(y,epsilon, "-o", markersize=3,label=f"mesh_{j}")
    Graph.legend()
    Graph.label("y [m]", "epsilon [m²/s³]")

#### Comparaison des différentes méthodes de calcul

In [ ]:
Graph=plot.Graph("Traceurs de la turbulence : comparaison des méthodes",nX=2, nY=nb_mesh,size=[8,6])
for j in range(nb_mesh):
    if nb_mesh==1:
        indice = 0
    else:
        indice = (0,j)
    Graph.addPlot(indice,f"mesh_{j}")
    for i in range(nb_method):
        file = file_pb_hydr
        if i>1:
            file = file_pb_multi
        Graph.addSegment(build+f"/{config[i]}/mesh_{j}/{file}_K.son", linestyle="solid", marker = 'o', markersize=3,label=f"{config[i]}")
    Graph.legend()
    Graph.label("y [m]", "k [m²/s²]")
    
for j in range(nb_mesh):
    if nb_mesh==1:
        indice = 1
    else:
        indice = (1,j)
    Graph.addPlot(indice,f"mesh_{j}")
    for i in range(nb_method):
        file = file_pb_hydr
        if i>1:
            file = file_pb_multi
        diss = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_DISS.son")[1::,1])
        k = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_K.son")[1::,1])
        if "tau" in config[i]:
            epsilon = 0.09*k/diss
        elif "omega" in config[i]:
            epsilon = 0.09*k*diss
        else:
            epsilon = diss
        y = np.linspace(y_min_prof[j], y_max_prof[j], nb_points_prof[j])
        Graph.add(y,epsilon, "-o", markersize=3,label=f"{config[i]}")
    Graph.legend()
    Graph.label("y [m]", "epsilon [m²/s³]")

### Perte de charge

In [ ]:
Re_Dh = 4*Re_H
Dh = 4*H
f_guess = 0.02
f = facteur_frottement(Re_Dh,f_guess) # coefficient de frottement donnée par la formule de Von Karman et Nikurade (Re>1e5)
v_m = inlet_velocity # vitesse moyenne sur une section
L = 100 # [m] : longueur du conduit
delta_p_estime = f*L/Dh*rho*v_m**2/2
print(f"Estimation de la perte de charge avec la formule de Von Karman et Nikuradse (comme le diagramme de Moody):\nDeltaP = {delta_p_estime:.1f} Pa")

In [ ]:
for j in range(nb_mesh):
    for i in range(nb_method):
        file = file_pb_hydr
        if i>1:
            file = file_pb_multi
        p_inlet = np.mean(np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_PRESSION_I.son")[1::,1]))
        p_outlet = np.mean(np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_PRESSION_O.son")[1::,1]))
        delta_p = p_outlet-p_inlet
        if "k-epsilon" in config[i]:
            delta_p = rho*delta_p
        #delta_u2 = u[1]**2-u[0]**2
        print(f"{config[i]}\t-\tmesh_{j}\t: deltaP = {delta_p:.1f} Pa\tEcart relatif : {100*(abs(delta_p)-delta_p_estime)/delta_p_estime:.0f}%")


### Pression
On verifie que le gradient de pression reste négligeable : dans ce cas la loi logarithmique doit bien être retrouvée.
Les valeurs maximales de p+ (paramètre de Mellor & Gibson) sont calculées.

In [ ]:
# # Plot P(x)

Graph=plot.Graph("Pressure evolution at y=0.8m",subtitle="pressure",nY=2,size=[15,7])
for i in range(nb_method_Pb_hydr):
    Graph.addPlot(i,f"{config[i]}")
    for j in range(nb_mesh):
        Graph.addSegment(build+f"/{config[i]}/mesh_{j}/{file_pb_hydr}_PRESSION.son",label=f"mesh_{j}")
    Graph.legend()
    Graph.label("x [m]", "P/rho [Pa.m³/kg]")
    
Graph=plot.Graph("Pressure gradient at y=0.8m",subtitle="pressure",nY=2,size=[15,7])
for i in range(nb_method_Pb_hydr):
    Graph.addPlot(i,f"{config[i]}")
    for j in range(nb_mesh):
        Graph.addSegment(build+f"/{config[i]}/mesh_{j}/{file_pb_hydr}_GRADP.son",label=f"mesh_{j}")
        Graph.legend()
        Graph.label("x [m]", "grad(P/rho) [m/s²]")
        dp_dx = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file_pb_hydr}_GRADP.son")[1::,1])
        p_plus = (mu/rho)*np.max(np.abs(dp_dx))/u_star_list[i][j]
        print(f"{config[i]}, mesh_{j} : p+={p_plus:.2e}")

In [ ]:
#fig=visit.Show(build+f"/VDF_k-epsilon/mesh_0/{file_pb_hydr}.lata","Pseudocolor","VITESSE_X_ELEM_dom")

# to add a list of viewing options
#fig.visuOptions(['no_databaseinfo','no_legend'])

# To display the visu with the initialization visit.Show()
#fig.plot()